In [28]:
import pandas as pd
import altair as alt

In [29]:
df_analysis_path = '/Volumes/arxivBeta/_Tobias/Opera/20250314/00_HarmonyEvaluation/20250314_Mitotic_Index.xls'

df = pd.read_excel(df_analysis_path)

In [30]:
df_monopol_path = '/Volumes/arxivBeta/_Tobias/Opera/20250314/20250318_Monopolar.xlsx'

df_monopol = pd.read_excel(df_monopol_path)

In [31]:
area_FOV = 1640961 # µm2
df["Population_density_cells_per_mm2"] = (df['Modified Nuclei - Number of Objects'] / area_FOV) * 1e6

In [32]:
df["Seeding_mm-2"] = df["Seeding_cm-2"] * 0.01

In [33]:
group_MI = df.groupby(["Cells", 'siRNA'])["Mitotic_index_relative"].mean()
group_MI

Cells  siRNA     
CM     KIF11_orig    2.147651
       LUC_orig      1.001819
IM     KIF11_orig    2.893552
       LUC_orig      1.000127
Name: Mitotic_index_relative, dtype: float64

In [34]:
group_MI2 = df.groupby(["Cells", 'siRNA'])["Mitotic Index (%)"].mean()
group_MI2

Cells  siRNA     
CM     KIF11_orig    1.189338
       LUC_orig      0.562893
IM     KIF11_orig    4.047886
       LUC_orig      1.393903
Name: Mitotic Index (%), dtype: float64

In [35]:
df.columns

Index(['Row', 'Column', 'Plane', 'Timepoint',
       'Modified Nuclei - Number of Objects',
       'Modified Nuclei - Intensity Region Alexa 647 Mean - Mean per Well',
       'Modified Nuclei - Intensity Region Alexa 647 Mean - StdDev per Well',
       'Modified Nuclei - Region Area [¬µm¬≤] - Mean per Well',
       'Modified Nuclei - Region Area [¬µm¬≤] - StdDev per Well',
       'Mitotic cells - Number of Objects', 'Mitotic Index (%)',
       'Number of Analyzed Fields', 'Height [¬µm]', 'Time [s]', 'Compound',
       'Concentration', 'Cells', 'Cell Count', 'Seeding_cm-2', 'siRNA',
       'Mitotic_index_relative', 'Population_density_cells_per_mm2',
       'Seeding_mm-2'],
      dtype='object')

In [36]:
colourscheme = "viridis"

In [37]:
def Binned_Boxplot(
    data, x, y, x_title, y_title, color, column, 
    Boxsize = 10,  
    Bin_Boxplot_width = 100, 
    Bin_Boxplot_height = 180):
    # Boxplot for binned x data
    BINNED_BOX = alt.Chart(
        data = data, 
        width = Bin_Boxplot_width, 
        height = Bin_Boxplot_height
    ).mark_boxplot(
        size = Boxsize, 
        extent = "min-max"
    ).encode(
    alt.X(x, title = None, axis = alt.Axis(grid = False)),
    alt.Y(y,  title = y_title, axis = alt.Axis(grid = False)),
    color = alt.Color(
        color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 15
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 10
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )
    return BINNED_BOX

In [38]:
def Binned_Boxplot2(
    data, x, y, x_title, y_title, y_min, y_max, color, column, 
    Boxsize = 10,  
    Bin_Boxplot_width = 50, 
    Bin_Boxplot_height = 180):
    # Boxplot for binned x data
    BINNED_BOX = alt.Chart(
        data = data, 
        width = Bin_Boxplot_width, 
        height = Bin_Boxplot_height
    ).mark_boxplot(
        size = Boxsize, 
        extent = "min-max"
    ).encode(
    alt.X(x, title = None, axis = alt.Axis(grid = False)),
    alt.Y(y,  title = y_title, axis = alt.Axis(grid = False), scale = alt.Scale(domain = [y_min, y_max])),
    color = alt.Color(
        color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 10
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 10
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )

    
    return BINNED_BOX

In [39]:
def Binned_Circles(
    data, x, y, x_title, y_title, color, column, 
    Boxsize = 200,  
    Bin_Boxplot_width = 75, 
    Bin_Boxplot_height = 180):
    # Boxplot for binned x data
    BINNED_BOX = alt.Chart(
        data = data, 
        width = Bin_Boxplot_width, 
        height = Bin_Boxplot_height
    ).mark_circle(
        size = Boxsize,
    ).encode(
    alt.X(x, title = None, axis = alt.Axis(grid = False)),
    alt.Y("mean(" + y + ")",  title = y_title, axis = alt.Axis(grid = False)),
    color = alt.Color(
        color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 15
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 15
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )    
    
    ERROR_Circles = alt.Chart(
            data = data
    ).mark_errorbar(extent = "stdev").encode(
        alt.X(x, title = x_title), 
        alt.Y("mean(" + y + ")", title = y_title), #scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 15
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 10
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )

    BINNED_Circle = BINNED_BOX
    return BINNED_Circle

In [40]:
def Scatterbin(dataframe, x, y, color, x_title, y_title, binextent, binstep, x_min, x_max, y_min, y_max,
            Circlesize = 10, 
            CircleOpacity = 0.4,  
            Scatter_width = 200, 
            Scatter_height = 200
               ):
    # Standard scatter plot 
    SCATTER = alt.Chart(
        data = dataframe, 
        width = Scatter_width, 
        height = Scatter_height
    ).mark_circle(
        opacity = CircleOpacity,
        size = Circlesize
    ).encode(
        alt.X(x, title = x_title, scale = alt.Scale(domain = [x_min, x_max])),
        alt.Y(y, title = y_title, scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    )
    
    SCATTER_bin = alt.Chart(
        data = dataframe, 
        width = Scatter_width, 
        height = Scatter_height
    ).mark_circle(
        opacity = 1,
        size = 150
    ).encode(
        alt.X(x, title = x_title, bin = alt.Bin(extent = binextent, step = binstep)), #scale = alt.Scale(domain = [x_min, x_max])),
        alt.Y("median(" + y + ")", title = y_title, bin = False), #scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    )
    
    ERROR_Scatterbin = alt.Chart(
            data = dataframe
    ).mark_errorbar(extent = "iqr").encode(
        alt.X(x, title = x_title, bin = alt.Bin(extent = binextent, step = binstep)), #scale = alt.Scale(domain = [x_min, x_max])),
        alt.Y("median(" + y + ")", title = y_title, bin = False), #scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    ) 

    
    SCATTERBIN = SCATTER + ERROR_Scatterbin + SCATTER_bin 
    return SCATTERBIN

In [41]:
SCATTERBIN_Seeding_Pop = Scatterbin(
    dataframe = df, 
    y = "Population_density_cells_per_mm2", 
    x = "Seeding_cm-2", 
    color = "siRNA", 
    y_title = "Population_density_cells_per_mm2", 
    x_title = "Seeding density cm-2", 
    binextent = [0, 60000], 
    binstep = 10000,
    y_min = 0,
    y_max = 2000,
    x_min = 0,
    x_max = 60000
)
SCATTERBIN_Seeding_Pop 

alt.LayerChart(...)

In [56]:
SCATTERBIN_SeedingMM_Pop = Scatterbin(
    dataframe = df[df["Cells"]=="CM"], 
    y = "Population_density_cells_per_mm2", 
    x = "Seeding_mm-2", 
    color = "siRNA", 
    y_title = "Population_density_cells_per_mm2", 
    x_title = "Seeding density mm-2", 
    binextent = [0, 700], 
    binstep = 100,
    y_min = 0,
    y_max = 2000,
    x_min = 0,
    x_max = 700
)
SCATTERBIN_SeedingMM_Pop 

alt.LayerChart(...)

In [43]:
BINBOX_MitoticIndex_Time_CM = Binned_Boxplot2(
    data = df[df["Cells"] == "CM"],
    x = 'siRNA',
    y = 'Mitotic Index (%)',
    x_title = '',
    y_title = 'Mitotic Index (%)',
    color = 'siRNA',
    column = 'Seeding_cm-2',
    y_min = 0,
    y_max = 6
)
BINBOX_MitoticIndex_Time_CM

alt.Chart(...)

In [44]:
BINBOX_MitoticIndex_Time_CMfold = Binned_Boxplot2(
    data = df[df["Cells"] == "CM"],
    x = 'siRNA',
    y = 'Mitotic_index_relative',
    x_title = '',
    y_title = 'Mitotic Index relative to control',
    color = 'siRNA',
    column = 'Seeding_cm-2',
    y_min = 0,
    y_max = 5
)
BINBOX_MitoticIndex_Time_CMfold 

alt.Chart(...)

In [45]:
BINBOX_MitoticIndex_Time_IM = Binned_Boxplot2(
    data = df[df["Cells"] == "IM"],
    x = 'siRNA',
    y = 'Mitotic Index (%)',
    x_title = '',
    y_title = 'Mitotic Index (%)',
    color = 'siRNA',
    column = 'Seeding_cm-2',
    y_min = 0,
    y_max = 6
)
BINBOX_MitoticIndex_Time_IM

alt.Chart(...)

In [46]:
BINBOX_MitoticIndex_Time_IMfold = Binned_Boxplot2(
    data = df[df["Cells"] == "IM"],
    x = 'siRNA',
    y = 'Mitotic_index_relative',
    x_title = '',
    y_title = 'Mitotic Index relative to control',
    color = 'siRNA',
    column = 'Seeding_cm-2',
    y_min = 0,
    y_max = 5
)
BINBOX_MitoticIndex_Time_IMfold 

alt.Chart(...)

In [47]:
BINBOX_MitoticIndex_Time_fold = Binned_Boxplot2(
    data = df,
    x = 'siRNA',
    y = 'Mitotic_index_relative',
    x_title = '',
    y_title = 'Mitotic Index relative to control',
    color = 'siRNA',
    column = 'Cells',
    y_min = 0,
    y_max = 5
)
BINBOX_MitoticIndex_Time_fold

alt.Chart(...)

In [48]:
BINBOX_MitoticIndex = Binned_Boxplot2(
    data = df,
    x = 'siRNA',
    y = 'Mitotic Index (%)',
    x_title = '',
    y_title = 'Mitotic Index (%)',
    color = 'siRNA',
    column = 'Cells',
    y_min = 0,
    y_max = 6
)
BINBOX_MitoticIndex

alt.Chart(...)

In [49]:
SCATTERBIN_MI_Pop = Scatterbin(
    dataframe = df, 
    x = "Modified Nuclei - Number of Objects", 
    y = "Mitotic Index (%)", 
    color = "Cells", 
    x_title = "Number of nuclei in FOV", 
    y_title = "Mitotic Index (%)", 
    binextent = [0, 4000], 
    binstep = 1000,
    x_min = 0,
    x_max = 4000,
    y_min = 0,
    y_max = 6
)
SCATTERBIN_MI_Pop

alt.LayerChart(...)

In [50]:
SCATTERBIN_MI_Pop_IM = Scatterbin(
    dataframe = df[df["Cells"] == "IM"], 
    x = "Population_density_cells_per_mm2", 
    y = "Mitotic Index (%)", 
    color = "siRNA", 
    x_title = "Cells per area (1/mm2)", 
    y_title = "Mitotic Index (%)", 
    binextent = [0, 2000], 
    binstep = 500,
    x_min = 0000,
    x_max = 2000,
    y_min = 0,
    y_max = 6
)
SCATTERBIN_MI_Pop_IM

alt.LayerChart(...)

In [51]:
SCATTERBIN_MI_Pop_CM = Scatterbin(
    dataframe = df[df["Cells"] == "CM"], 
    x = "Population_density_cells_per_mm2", 
    y = "Mitotic Index (%)", 
    color = "siRNA", 
    x_title = "Cells per area (1/mm2)", 
    y_title = "Mitotic Index (%)", 
    binextent = [0, 2000], 
    binstep = 500,
    x_min = 0000,
    x_max = 2000,
    y_min = 0,
    y_max = 6
)
SCATTERBIN_MI_Pop_CM

alt.LayerChart(...)

In [52]:
SCATTERBIN_MI_Pop2_CM = Scatterbin(
    dataframe = df[df["Cells"] == "CM"], 
    x = "Seeding_cm-2", 
    y = "Mitotic Index (%)", 
    color = "siRNA", 
    x_title = "Seeding (1/cm2)", 
    y_title = "Mitotic Index (%)", 
    binextent = [0, 60000], 
    binstep = 10000,
    x_min = 0,
    x_max = 60000,
    y_min = 0,
    y_max = 6
)
SCATTERBIN_MI_Pop2_CM

alt.LayerChart(...)

In [53]:
BINBOX_MitoticIndex_Time_Targets = Binned_Circles(
    data = df[df["Cells"] == "IM"],
    column = 'Seeding_cm-2',
    y = 'Mitotic Index (%)',
    x_title = '',
    y_title = 'Mitotic Index (%)',
    color = 'Cells',
    x = 'siRNA'
)
#BINBOX_MitoticIndex_Time_Targets

In [54]:
BINBOX_Phenotype = Binned_Boxplot2(
    data = df_monopol,
    x = 'siRNA',
    y = 'Percentage_of_total',
    x_title = '',
    y_title = 'Percentage_of_total',
    color = 'siRNA',
    column = 'Cells',
    y_min = 0,
    y_max = 100
)
BINBOX_Phenotype

alt.Chart(...)